In [15]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [16]:
# Import location files
# File to Load 
csv_file_geo = "Resources/countries_geo.csv"
geo_df = pd.read_csv(csv_file_geo)
geo_df.head()

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [17]:
#drop the current country column that is not valuable
geo_drop = geo_df.drop(columns=['country'])
geo_drop.head()

,latitude,longitude,name
0,42.546245,1.601554,Andorra
1,23.424076,53.847818,United Arab Emirates
2,33.939110,67.709953,Afghanistan
3,17.060816,-61.796428,Antigua and Barbuda
4,18.220554,-63.068615,Anguilla


In [18]:
#change "name" to "country" to prepare for the matching ID
geo_renamed = geo_drop.rename(columns={"name": "country"})
geo_renamed.head()

,latitude,longitude,country
0,42.546245,1.601554,Andorra
1,23.424076,53.847818,United Arab Emirates
2,33.939110,67.709953,Afghanistan
3,17.060816,-61.796428,Antigua and Barbuda
4,18.220554,-63.068615,Anguilla


In [22]:
# Import continent files
# File to Load 
csv_file_con = "Resources/continent.csv"
con_df = pd.read_csv(csv_file_con)
con_df.head()

,country,code,country-code,sub-region,intermediate-region,region-code
0,Afghanistan,AF,AFG,Asia,Southern Asia,NaN
1,Aland Islands,AX,ALA,Europe,Northern Europe,NaN
2,Albania,AL,ALB,Europe,Southern Europe,NaN
3,Algeria,DZ,DZA,Africa,Northern Africa,NaN
4,American Samoa,AS,ASM,Oceania,Polynesia,NaN


In [23]:
#change "name" to "country" to prepare for the matching ID
con_renamed = con_df.rename(columns={"name": "country"})
con_renamed.head()

,country,code,country-code,sub-region,intermediate-region,region-code
0,Afghanistan,AF,AFG,Asia,Southern Asia,NaN
1,Aland Islands,AX,ALA,Europe,Northern Europe,NaN
2,Albania,AL,ALB,Europe,Southern Europe,NaN
3,Algeria,DZ,DZA,Africa,Northern Africa,NaN
4,American Samoa,AS,ASM,Oceania,Polynesia,NaN


In [24]:
#merge location detail files based on country
loc_info = pd.merge(geo_renamed,
                  con_renamed,
                  on='country')
loc_info.head()

,latitude,longitude,country,code,country-code,sub-region,intermediate-region,region-code
0,42.546245,1.601554,Andorra,AD,AND,Europe,Southern Europe,NaN
1,23.424076,53.847818,United Arab Emirates,AE,ARE,Asia,Western Asia,NaN
2,33.939110,67.709953,Afghanistan,AF,AFG,Asia,Southern Asia,NaN
3,17.060816,-61.796428,Antigua and Barbuda,AG,ATG,Americas,Latin America and the Caribbean,Caribbean
4,18.220554,-63.068615,Anguilla,AI,AIA,Americas,Latin America and the Caribbean,Caribbean


In [27]:
#rearrange the order of columns
loc_info = loc_info[['country', 'latitude', 'longitude', 'code','country-code','sub-region','intermediate-region']]
loc_info.head()

,country,latitude,longitude,code,country-code,sub-region,intermediate-region
0,Andorra,42.546245,1.601554,AD,AND,Europe,Southern Europe
1,United Arab Emirates,23.424076,53.847818,AE,ARE,Asia,Western Asia
2,Afghanistan,33.939110,67.709953,AF,AFG,Asia,Southern Asia
3,Antigua and Barbuda,17.060816,-61.796428,AG,ATG,Americas,Latin America and the Caribbean
4,Anguilla,18.220554,-63.068615,AI,AIA,Americas,Latin America and the Caribbean


In [28]:
# Connect to local Database
USERNAME = "postgres"
PASSWORD = "099426"
DATABASE = "postgres"
rds_connection_string = f"{USERNAME}:{PASSWORD}@localhost:5432/{DATABASE}"
print(rds_connection_string)
engine = create_engine(f'postgresql://{rds_connection_string}')

postgres:099426@localhost:5432/postgres


In [29]:
# Check for tables
engine.table_names()

['loc']

In [ ]:
# Use pandas to load csv converted DataFrame into database
loc_info.to_sql(name='loc', con=engine, if_exists='append', index=False)

# check for loaded data
pd.read_sql_query('select * from loc', con=engine).head()

In [31]:
loc_info.to_csv('loc.csv')